In [1]:
from pylogic import *
from pylogic.proposition.proof_search import proof_search, _BackwardProver

In [2]:
# Example usage
A = propositions("A", is_assumption=True)
B, C, D, E, F, G = propositions("B", "C", "D", "E", "F", "G")
p6 = Implies(And(Or(D, E), Or(A, F)), G, is_assumption=True)
A_imp_B = Implies(A, B, is_assumption=True)
B_imp_C = Implies(B, C, is_assumption=True)
C_imp_D = Implies(C, D, is_assumption=True)
kb = [
    # A,
    # A_imp_B,
    # B_imp_C,
    # C_imp_D,
    C.implies(G).assume(),
    B.implies(F).assume(),
    B.or_(C).assume(),
    # p6,
]
target = F.or_(G)
proof = proof_search(kb, target)
print(proof)


F \/ G


In [3]:
P, Q, R, S, T = propositions("P", "Q", "R", "S", "T")
p1 = P.implies(Q).and_(R.implies(S)).assume()
p2 = Q.or_(S).implies(T).assume()
p3 = P.or_(R).assume()
p4 = neg(T).assume()

target = neg(P).and_(neg(R))
kb = [
    p1,
    p2,
    p3,
    p4,
]
proof = proof_search(kb, target)
print(proof.deduced_from)

('ex_falso', Proposition(contradiction))


In [4]:
A, B = propositions("A", "B")
p1 = A.or_(B).assume()
p2 = neg(B).assume()

target = A
kb = [
    p1,
    p2,
]
proof = proof_search(kb, target)
print(proof.deduced_from)

('by_cases', Or(Proposition(A), Proposition(B)), 'has inner contexts')


In [5]:
A, B, C, D, E, F = propositions("A", "B", "C", "D", "E", "F")
settings["USE_CLASSICAL_LOGIC"] = True

# Premises
p1 = A.implies(B.and_(F)).assume()
p2 = D.implies(neg(B)).assume()
p3 = C.or_(D).assume()
p4 = neg(E).implies(neg(D)).assume()

# Target
target = A.implies(C)

kb = [
    p1,
    p2,
    p3,
    # p4,
]

proof = proof_search(kb, target)
print(proof.deduced_from)


('by_cases', Or(Proposition(C), Proposition(D)), 'has inner contexts')


In [6]:
I, J, K, M, N = propositions("I", "J", "K", "M", "N")
p1 = I.implies(J).assume()
p2 = J.or_(I, K).assume()
p3 = J.implies(M).assume()
p4 = neg(M).assume()
p5 = K.implies(neg(N)).assume()

target = neg(N)
kb = [
    p1,
    p2,
    p3,
    p4,
    p5,
]

In [7]:
_BackwardProver(kb)

In [8]:

proof = proof_search(kb, target)
print(proof.deduced_from)

('modus_ponens', Proposition(K), Implies(Proposition(K), Not(Proposition(N))))


In [9]:
letters = ["A",  "B", "E", "K"]; vowels = ["A", "E"]
colors = ["blue", "red"]

face_up = ["E", "K", "blue", "red"]

cards = [Constant(f"card_{i}") for i in range(len(face_up))]
is_vowel, is_blue = predicates("is_vowel", "is_blue")

def condition(card):
    return is_vowel(card).implies(is_blue(card))

kb = []
targets = [condition(cards[i]) for i in range(len(face_up))]
for i,c in enumerate(face_up):
    if c in vowels:
        kb.append(is_vowel(cards[i]).assume())
    elif c == "blue":
        kb.append(is_blue(cards[i]).assume())
    elif c == "red":
        kb.append(neg(is_blue(cards[i])).assume())
    else:
        kb.append(neg(is_vowel(cards[i])).assume())

print(*kb)

for i,t in enumerate(targets):
    try:
        proof = proof_search(kb, t)
        print(f"Card {i} satisfies the condition")
    except ValueError:
        print(f"Card {i} may not satisfy the condition")


is_vowel(card_0) ~is_vowel(card_1) is_blue(card_2) ~is_blue(card_3)
Card 0 may not satisfy the condition
Card 1 satisfies the condition
Card 2 satisfies the condition
Card 3 may not satisfy the condition


In [10]:
is_married, is_looking_at = predicates(r"is\_married", r"is\_looking\_at")

A, B, C, D = people = constants("A", "B", "C", "D")

looking_at = {
    (A, B): True,
    (D, C): True,
}

def iff(a, b):
    return a.implies(b).and_(b.implies(a))

def married_or_not(x):
    return is_married(x).or_(neg(is_married(x)))

def condition(x, y):
    return is_married(x).and_(neg(is_married(y)), is_looking_at(x, y))

kb = assume(
    is_looking_at(A, B),
    is_looking_at(D, C),
    is_married(A),
    iff(is_married(B), is_married(D)),
    married_or_not(B),
    married_or_not(D),
    neg(is_married(C)),
)

target_components = []
for x in people:
    for y in people:
        if looking_at.get((x, y), False):
            target_components.append(condition(x, y))

target = target_components[0].or_(*target_components[1:])
proven_target = proof_search(kb, target)
display(proven_target)
print(proven_target.is_proven)

Or(And(Proposition(is\_married, A), Not(Proposition(is\_married, B)), Proposition(is\_looking\_at, A, B)), And(Proposition(is\_married, D), Not(Proposition(is\_married, C)), Proposition(is\_looking\_at, D, C)))

True


In [11]:
A, B = propositions("A", "B")
kb = []

target = B.implies(neg(A).implies(B))

proof = proof_search(kb, target)
print(proof.deduced_from)

('close_assumptions_context', 'has inner contexts')


In [12]:

P, Q, R, S = propositions("P", "Q", "R", "S")

p1 = neg(P.and_(Q.or_(R, S)))#.assume()
p2 = neg(P).or_(neg(Q).and_(neg(R), neg(S))).assume()

kb = [
    p2,
]

target = p1
proof = proof_search(kb, target)
print(proof.deduced_from)

('by_cases', Or(Not(Proposition(P)), And(Not(Proposition(Q)), Not(Proposition(R)), Not(Proposition(S)))), 'has inner contexts')


In [13]:
P, Q, R, S = propositions("P", "Q", "R", "S")
p1 = neg(P.and_(Q)).assume()
p2 = neg(P).or_(neg(Q))#.assume()
kb = [
    p1,
]
target = p2
proof = proof_search(kb, target)
print(proof.deduced_from)


ValueError: No rule found to prove ~P \/ ~Q

In [14]:
P, Q, R, S = propositions("P", "Q", "R", "S")
p1 = neg(P.and_(Q))#.assume()
p2 = neg(P).or_(neg(Q)).assume()
kb = [
    p2,
]
target = p1
proof = proof_search(kb, target)
print(proof.deduced_from)


('by_cases', Or(Not(Proposition(P)), Not(Proposition(Q))), 'has inner contexts')


In [15]:
P, Q, R, S = propositions("P", "Q", "R", "S")

p1 = P.assume()
kb = [
    p1,
]
target = neg(neg(P))
proof = proof_search(kb, target)
print(proof.deduced_from)

None


In [16]:
from pylogic.enviroment_settings.settings import settings

settings["USE_CLASSICAL_LOGIC"] = False
P, Q, R, S = propositions("P", "Q", "R", "S")

p1 = neg(neg(P)).assume()
kb = [
    p1,
]
target = P
proof = proof_search(kb, target)
print(proof.deduced_from)

ValueError: No rule found to prove P